In [1]:
!git clone https://github.com/ruzanovad/diploma.git

Cloning into 'diploma'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 418 (delta 230), reused 329 (delta 141), pack-reused 0 (from 0)
Receiving objects: 100% (418/418), 28.94 MiB | 31.03 MiB/s, done.
Resolving deltas: 100% (230/230), done.


In [4]:
%cd diploma/cnn-rnn

[Errno 2] No such file or directory: 'diploma/cnn-rnn'
/kaggle/working/diploma


In [ ]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

OSError: /usr/local/lib/python3.10/dist-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [9]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

NameError: name 'Path' is not defined